In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' 

In [2]:
import torch
import torch.nn as nn
from d2l import torch as d2l

In [5]:
# 6.2.1 二维互相关运算
def corr2d(X, K): #@save
    """计算二维互相关运算。"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [6]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]], dtype=torch.float32)
K = torch.tensor([[0,1],[2,3]], dtype=torch.float32)
Y = corr2d(X, K)
print(Y)

tensor([[19., 25.],
        [37., 43.]])


In [7]:
# 6.2.2 卷积层
class Conv2D(nn.Module):
    def __init__(self,  kernel_size):
        super(Conv2D, self).__init__()
        self.kernel_size = kernel_size
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zero_(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [8]:
# 6.2.3 图像中目标的边缘检测

X = torch.ones((6,8))
X [:,2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


In [10]:
K = torch.tensor([[1.0,-1.0]], dtype=torch.float32)

Y = corr2d(X, K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [11]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [14]:
# 6.2.4 学习卷积核
# 构造一个二维的卷积层，它具有1个输出通道和形状为（1,2)的卷积核。
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
# 卷积核的权重张量的形状为（1,1,2,1），即（批量大小，输入通道，卷积核高度，卷积核宽度）。
# 其中批量大小和输入通道都是1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

lr = 3e-2
num_epochs = 20

for epoch in range(num_epochs):
    # 前向传播
    Y_hat = conv2d(X)
    # 计算损失
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    # 反向传播
    loss.sum().backward()
    # 更新参数
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    # 打印损失
    if (epoch + 1) % 2 == 0:
        print(f"epoch {epoch + 1}, loss {loss.sum():.3f}")

epoch 2, loss 3.218
epoch 4, loss 0.562
epoch 6, loss 0.103
epoch 8, loss 0.021
epoch 10, loss 0.005
epoch 12, loss 0.001
epoch 14, loss 0.000
epoch 16, loss 0.000
epoch 18, loss 0.000
epoch 20, loss 0.000


In [15]:
print(conv2d.weight.data.reshape((1,2)))

tensor([[ 0.9993, -1.0005]])
